In [1]:
import os, urllib.request, json, shapefile, shutil, subprocess, csv, fiona, shapely, geopandas, psycopg2, pandas
from osgeo import gdal, ogr, osr
from datetime import datetime

In [2]:
os.environ['SHAPE_ENCODING'] = "utf-8"
basePath = 'C:\\saprog\\projeto'
outputPath = 'C:\\saprog\\projeto\\output'
os.chdir(basePath)

def deleteAndCreateOutput():
    print ('A limpar e criar directoria de outputs...')
    outDir = os.path.join(basePath, 'output')
    if os.path.isdir(outDir):
        shutil.rmtree(outDir)
    os.mkdir(outDir)

deleteAndCreateOutput()

A limpar e criar directoria de outputs...


In [3]:
print('A ler shapefile...')
caop = geopandas.read_file('caop.shp',encoding='utf-8')
# caop.plot()
# caop.head()
# gdf = caop.loc[caop['Concelho'] == 'Bombarral']
# gdf.head()

A ler shapefile...


In [4]:
print('A executar Dissolve...')
distritos = caop.dissolve(by = 'Distrito')
# distritos.plot()
# distritos.head(1)

A executar Dissolve...


In [5]:
print('A extrair centroides das regiões admninistrativas...')
shpOutput = os.path.join(outputPath, 'centroides.shp')

# dataProj = distritos.to_crs(epsg = 3763)
# centroides = dataProj.centroid
centroides = distritos.centroid
centroides.to_file(shpOutput, encoding='utf-8')
# centroides.plot()
# centroides.head()

A extrair centroides das regiões admninistrativas...
<ipython-input-5-5a8a41490fe0>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroides = distritos.centroid


In [6]:
coordx = centroides.x.to_dict()
coordy = centroides.y.to_dict()
# print(coordx)
# print(coordy)

def getCoordinates(dicX, dicY):
    coord = [dicX, dicY]
    coordDic = {}
    for i in dicX.keys():
        coordDic[i] = tuple(coordDic[i] for coordDic in coord)
    return coordDic

print('A extrair coodenadas...')
coord = getCoordinates(coordx, coordy)

A extrair coodenadas...
{'Aveiro': (-8.46838037713987, 40.7236925098494), 'Beja': (-7.94388147040214, 37.82970078780623), 'Braga': (-8.309815844734775, 41.55294943650094), 'Bragança': (-6.859336412433179, 41.509404985269676), 'Castelo Branco': (-7.5016170094379, 39.946483706164535), 'Coimbra': (-8.33590045610553, 40.204414004264905), 'Faro': (-8.13168379460421, 37.24368427672163), 'Guarda': (-7.229585420073208, 40.641320980615), 'Leiria': (-8.775241866061927, 39.717272858669155), 'Lisboa': (-9.163514891800212, 38.9967494344431), 'Portalegre': (-7.620439565448195, 39.190063045702445), 'Porto': (-8.352739071777673, 41.22479245090154), 'Santarém': (-8.477568721428257, 39.293494949281204), 'Setúbal': (-8.652857331305352, 38.318451316051366), 'Viana do Castelo': (-8.50709117586541, 41.87785654627992), 'Vila Real': (-7.631700989514108, 41.55496982213632), 'Viseu': (-7.870908401501086, 40.79887555573913), 'Évora': (-7.8417966523604, 38.603868659253635)}


In [7]:
# Carregamento de shapefile na bd
# command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
#           "-f", "PostgreSQL",
#           "PG:host=localhost user=postgres dbname=meteo password=3763",
#           shpOutput,
#           "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
#           " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "centroides", "-overwrite"]
# subprocess.check_call(command)

In [9]:
apiKey = '44cfad7f82ec61d3f420de3201b703d4'

def getWeather(coordDic, apikey):
    forecast = []
    for item in coord.items():
        lat = str(item[1][1])
        long = str(item[1][0])
        url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + long + \
            '&exclude=minutely,hourly,daily,alerts&appid=' + apiKey + '&units=metric'
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            districtForecast = {}
            districtForecast['distrito'] = item[0]
            districtForecast['date'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%d-%m-%Y')
            districtForecast['time'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%H:%M:%S')
            main = data.get('current').get('weather')
            for item in main:
                districtForecast['weather_desc'] = item.get('main')
            districtForecast['temperature'] = data.get('current').get('temp')
            districtForecast['feels_like'] = data.get('current').get('feels_like')
            districtForecast['pressure'] = data.get('current').get('pressure')
            districtForecast['humidity'] = data.get('current').get('humidity')
            districtForecast['dew_point'] = data.get('current').get('dew_point')
            districtForecast['ultrav_index'] = data.get('current').get('uvi')
            districtForecast['wind_speed'] = data.get('current').get('wind_speed')
            districtForecast['wind_deg'] = data.get('current').get('wind_deg')


            forecast.append(districtForecast)
    return forecast

df = pandas.DataFrame(getWeather(coord, apiKey))
df.head(-1)

,distrito,date,time,weather_desc,temperature,feels_like,pressure,humidity,dew_point,ultrav_index,wind_speed,wind_deg
0,Aveiro,14-03-2021,15:42:18,Clouds,15.30,10.38,1027,72,10.29,1.44,7.20,340
1,Beja,14-03-2021,15:42:19,Clear,19.00,15.16,1024,37,4.00,1.65,3.60,30
2,Braga,14-03-2021,15:42:19,Clouds,17.39,16.34,1026,50,6.87,1.38,0.45,10
3,Bragança,14-03-2021,15:42:20,Clouds,14.38,12.58,1025,52,4.66,1.15,0.87,320
4,Castelo Branco,14-03-2021,15:42:20,Clear,17.48,14.10,995,48,6.36,1.52,3.63,44
5,Coimbra,14-03-2021,15:42:20,Clear,16.84,15.29,1027,74,12.18,1.47,3.17,340
6,Faro,14-03-2021,15:42:20,Clear,19.00,15.74,1023,45,6.81,1.72,3.60,120
7,Guarda,14-03-2021,15:42:21,Clear,17.21,14.06,1026,47,5.81,1.38,3.13,76
8,Leiria,14-03-2021,15:42:21,Clear,17.28,11.61,1026,63,10.18,1.57,8.23,340
9,Lisboa,14-03-2021,15:42:21,Clear,18.18,16.25,1023,53,8.46,1.69,2.24,39
